In [1]:
import os
import time

In [18]:
from cozepy import COZE_CN_BASE_URL

# Get an access_token through personal access token or oauth.
coze_api_token = "pat_3xeECicYAnAijwCVUaeMuZEvN4WLimnAfrmu9HApdw5amSPRsu142pqi49RLUVZV" # os.getenv("COZE_API_TOKEN")
# The default access is api.coze.com, but if you need to access api.coze.cn,
# please use base_url to configure the api endpoint to access
coze_api_base = os.getenv("COZE_API_BASE") or COZE_CN_BASE_URL

In [3]:
from cozepy import Coze, TokenAuth, Message, ChatStatus, MessageContentType  # noqa


In [4]:
# Init the Coze client through the access_token.
coze = Coze(auth=TokenAuth(token=coze_api_token), base_url=coze_api_base)

# Create a bot instance in Coze, copy the last number from the web link as the bot's ID.
bot_id = os.getenv("COZE_BOT_ID") or "7471918474705747977"
# The user id identifies the identity of a user. Developers can use a custom business ID
# or a random string.
user_id = "macbook"


In [5]:
    # Call the coze.chat.create method to create a chat. The create method is a non-streaming
    # chat and will return a Chat class. Developers should periodically check the status of the
    # chat and handle them separately according to different states.
chat = coze.chat.create(
        bot_id=bot_id,
        user_id=user_id,
        additional_messages=[
            # Message.build_user_question_text("Who are you?"),
            # Message.build_assistant_answer("I am Bot by Coze."),
            Message.build_user_question_text("煎虾时应该先给锅放油还是先放虾？"),
        ],
    )


In [6]:
    # Assume the development allows at most one chat to run for 10 minutes. If it exceeds 10
    # minutes, the chat will be cancelled.
    # And when the chat status is not completed, poll the status of the chat once every second.
    # After the chat is completed, retrieve all messages in the chat.
start = int(time.time())
timeout = 600
while chat.status == ChatStatus.IN_PROGRESS:
        if int(time.time()) - start > timeout:
            # too long, cancel chat
            coze.chat.cancel(conversation_id=chat.conversation_id, chat_id=chat.id)
            break

        time.sleep(1)
        # Fetch the latest data through the retrieve interface
        chat = coze.chat.retrieve(conversation_id=chat.conversation_id, chat_id=chat.id)


In [7]:
# When the chat status becomes completed, all messages under this chat can be retrieved through the list messages interface.
messages = coze.chat.messages.list(conversation_id=chat.conversation_id, chat_id=chat.id)
for message in messages:
        print(f"role={message.role}, content={message.content}")

role=MessageRole.ASSISTANT, content={"msg_type":"knowledge_recall","data":"{\"chunks\":[{\"slice\":\"煎虾之前，先用小苏打清洗，一方面可以适当去除孔雀石绿这种公害物质，而另一方 面碱性环境可以加速美拉德反应。\\n很多同学照着我提供的菜谱做虾，总是煎不好，就是觉得洗个虾没什么难的，忽略了 小苏打在美拉德反应中的作用。\\n我们把锅烧热后，倒高烟点的油，铺满锅底，这时候的油是热能的介质，纯脂肪是不 参与美拉德反应的（可催化，但不参与）。\\n*高烟点油，简单讲就是不容易烧出烟的油，冒烟就意味着油质开始恶化。烟点最高的 是牛油果油与茶籽油，大约在 250～270 度，家庭使用压榨菜籽油就可以了，190～220 度左右。黄油烟点垫底（120～150），所以不能早放，没有达到美拉德反应温度时就开 始冒烟。详见 ： http://t.cn/A6pndFvM\\n把虾务必要擦干，不仅为了不爆锅，而且也是为了快速进入到美拉德反应的水分比中。\\n下虾一个一个下，记住顺序，其实凭颜色也能判断。这时候虾皮会迅速产生美拉德反 应，厨房里肯定充满了焦香。\\n虾煎到一面发红，肉呈现半白色，注意，此时其实可以转到中火，以便腾出更多时间 来翻面，此时虾的水分已经蒸发，锅温大致恒定在 140～160 度之间。这是美拉德反应 最好的温度条件。\\n*黄油：这个时候按照顺序翻面，加入黄油。黄油与植物油相比较，含有一定量的蛋白 质、氨基酸与糖，自身就能发生美拉德反应，遇热后成为液体附着在食材上，让食材 全面美拉德化。\\n不能过早加入黄油是因为黄油遇热容易产生大分子的褐色蛋白，等你一个个翻面之后 就可能焦糊，除非你技巧娴熟到一颠锅就能让所有的虾翻面。\\n加入黄油后保持中火，再煎熟另一面。你会发现煎虾的香味升级了，这是因为黄油内 的蛋白质也参与到美拉德反应中，而且黄油是液体，能渗透到虾壳与虾肉里。\\n此时美拉德反应最迅速，也是最充分的时候。具备了温度、蛋白质、水分比等所有最 优条件。\",\"score\":0.98046875,\"meta\":{\"dataset\":{\"id\":\"7471915812086235174\",\"name\":\"美拉德反应\"},\"document\"

In [8]:
print(messages[1].content)

煎虾时应该先给锅放油。具体操作是把锅烧热后，倒高烟点的油，铺满锅底，之后把擦干的虾一个一个下锅。  


In [11]:
messages[3]

Message(role=<MessageRole.ASSISTANT: 'assistant'>, type=<MessageType.FOLLOW_UP: 'follow_up'>, content='煎虾的过程中需要翻面吗？', content_type=<MessageContentType.TEXT: 'text'>, meta_data=None, id='7474154147894296602', conversation_id='7474154121335963658', section_id=None, bot_id='7471918474705747977', chat_id='7474154121335980042', created_at=1740212122, updated_at=1740212121)

In [17]:
messages_answer = ""
messages_follow_up = []

for message in messages:
    if message.type == "answer":
        messages_answer = message.content
    elif message.type == "follow_up":
        messages_follow_up.append(message.content)

print(messages_answer)
print(messages_follow_up)


煎虾时应该先给锅放油。具体操作是把锅烧热后，倒高烟点的油，铺满锅底，之后把擦干的虾一个一个下锅。  
['煎虾的过程中需要翻面吗？', '煎虾的时间一般是多久？', '煎虾时可以加一些配菜吗？']
